# Visualize a Box Trap

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mplcolor

import re,glob,os,sys
import dgutils.colors as colortools
from collections import defaultdict
from vapory import *
π = np.pi
from PIL import Image


%matplotlib inline
%config InlineBackend.figure_format = 'svg'
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

included = ["colors.inc","textures.inc","functions.inc"]
filename = "../Figures/Summary/box_trap.png"

## Setup colors and scene

In [12]:
blue = mplcolor.to_rgb('#0073CD')
blue = mplcolor.to_rgb('#006C93')
grey = mplcolor.to_rgb('#9D9FAB')
brown = mplcolor.to_rgb('#935C36')
green = mplcolor.to_rgb('#15A05E')
orange = mplcolor.to_rgb('#E65933')
yellow = mplcolor.to_rgb('#EBEA64')
smokey = mplcolor.to_rgb('#58595B')
legacy = mplcolor.to_rgb('#579584')
regalia = mplcolor.to_rgb('#754A7E')
energy = mplcolor.to_rgb('#EE3E80')
leconte = mplcolor.to_rgb('#8D2048')
red = mplcolor.to_rgb('#8B0000')
river = mplcolor.to_rgb('#517C96')

In [13]:
L = np.array([20,20,20])

cam = Camera('location',[25,5,-28],'look_at',[0,0,0])
bg = Background("color", "White",'transmit',1.0)
lights = [LightSource([25,5,-28], 'color','White','shadowless')]
lights.extend([LightSource([100,0,0], 'color','White')])
lights.extend([LightSource([0,0,0], 'color','White shadowless')])
lights.extend([LightSource([0,100,0], 'color','White')])

col_atom = Texture(Finish('phong','0.0'),Pigment('color',blue,'transmit',0.0))

col_boxg = Texture(Finish('phong','0.0'),Pigment('color',grey,'transmit',0.0))
col_box = Texture(Finish('phong','0.0'),Pigment('color','White','transmit',0.4))
col_boxr = Texture(Finish('phong','0.0'),Pigment('color','White','transmit',0.3))

col_spin = Texture(Finish('phong','0.3'),Pigment('color',yellow,'transmit',0.0))

col_level0 = Texture(Finish('phong','0.5'),Pigment('color',smokey,'transmit',0.0))
col_levelp = Texture(Finish('phong','0.5'),Pigment('color',legacy,'transmit',0.0))
col_levelm = Texture(Finish('phong','0.5'),Pigment('color',energy,'transmit',0.0))
col_level = [col_levelm,col_level0,col_levelp]

col_hot = Texture(Finish('phong','0.5'),Pigment('color',red,'transmit',0.3))
col_cold = Texture(Finish('phong','0.5'),Pigment('color',river,'transmit',0.3))

δ = 0.1

## Draw the Box Trap

In [14]:
# 4 separate panels
#box = [Box([L[0]/2,-L[1]/2-δ,-L[2]/2],[L[0]/2+δ,L[1]/2+δ,L[2]/2],col_boxr)]
#box.extend([Box([-L[0]/2-δ,-L[1]/2,-L[2]/2],[-L[0]/2,L[1]/2,L[2]/2],col_boxr)])
#box.extend([Box([-L[0]/2-δ,-L[1]/2-δ,-L[2]/2],[L[0]/2+δ,-L[1]/2,L[2]/2],col_box)])
#box.extend([Box([-L[0]/2-δ,L[1]/2,-L[2]/2],[L[0]/2+δ,L[1]/2+δ,L[2]/2],col_box)])
#box.extend([Box([-L[0]/2-δ,-L[1]/2-δ,L[2]/2],[L[0]/2+δ,L[1]/2+δ,L[2]/2+δ],col_box)])

box = [Box([-L[0]/2-δ,-L[1]/2-δ,-L[2]/2-δ],[L[0]/2+δ,L[1]/2+δ,L[2]/2+δ])]
box.append(Box([-L[0]/2,-L[1]/2,-L[2]/2-2],[L[0]/2,L[1]/2,L[2]/2]))
box = [Difference(*box,col_boxr)]

## Fill With Particles

In [25]:
r_core = 1
num_particles = 250

pos = []

# add the initial particle
pos.append(-0.5*L + r_core + (L-1.25*r_core)*np.random.random(3))

num_inserted = len(pos)
num_attempts = 0
max_num_attempts = 1E6

while num_inserted < num_particles and num_attempts < max_num_attempts:
    num_attempts += 1
    
    trial_pos = -0.5*L + r_core + (L-1.25*r_core)*np.random.random(3)
    
    accept = True
    for cpos in pos:
        rsep = np.linalg.norm(trial_pos - cpos)
        if rsep < 2*r_core:
            accept = False
            break
    if accept:
        pos.append(trial_pos)
        num_inserted += 1
        
    if num_attempts == max_num_attempts:
        print('Reached maximum number of attempts!')

pos = np.array(pos)
sphere = [Sphere(cpos, 0.35*r_core, col_atom) for cpos in pos]


## Add the Spins

In [26]:
cylinder = []
cone = []
for cpos in pos:
    θ = 2*π*np.random.random()
    ϕ = np.arccos(2*np.random.random()-1.0);
    rspin = 0.55*r_core*np.array([np.sin(θ)*np.cos(ϕ),np.sin(θ)*np.sin(ϕ),np.cos(ϕ)])
    if np.random.random() < 0.5:
        direction = 1
    else:
        direction = -1
    
        
    cylinder.append(Cylinder(cpos-rspin,cpos+rspin,0.07,col_spin))

    
    head_start = cpos + direction*rspin
    head_end = cpos + direction*1.5*rspin
        
    cone.append(Cone(head_start,0.2,head_end,0.0,col_spin))

## Add the energy levels

Split up the degenerate states into subgroups

In [27]:
from itertools import groupby
ε = []
for nx in range(1,4):
    for ny in range(1,4):
        for nz in range(1,4):
            ε.append(nx**2 + ny**2 + nz**2)
ε = np.sort(ε)[:17]-np.min(ε)
ε = np.array(ε)/(np.max(ε)/11)
ε_levels = [list(group) for key, group in groupby(ε)]

levels = []
δlevel = 0.2

for clevel in ε_levels:
    g = len(clevel)
    Δ = 6/g
    for j,cε in enumerate(clevel):
        for i,δlevel in enumerate([-0.125,0,0.125]):
            level_start = [L[0]/2+δ,-2.5+cε+δlevel,-L[2]/2+2+j*Δ]
            level_end = [L[0]/2+δ,-2.5+cε+δlevel,-L[2]/2+2+(j+1)*Δ-2*δ]
            levels.append(Cylinder(level_start,level_end,0.05,col_level[i]))

### This is the old way, sufficient if we don't have degenerate levels

In [18]:
# ε = []
# eL = [np.sqrt(82),np.sqrt(120),np.sqrt(99)]
# for nx in range(3):
#     for ny in range(3):
#         for nz in range(3):
#             ε.append(nx**2/eL[0]**2 + ny**2/eL[1]**2 + nz**2/eL[2]**2)
# ε = np.unique(ε)
# ε = np.sort(ε)[:8]
# ε /= (np.max(ε)/11)

# levels = []
# δlevel = 0.2
# for cε in ε:
#     for i,δlevel in enumerate([-0.125,0,0.125]):
#         level_start = [L[0]/2+δ,-2.5+cε+δlevel,-L[2]/2+2]
#         level_end = [L[0]/2+δ,-2.5+cε+δlevel,-L[2]/2+8]
#         levels.append(Cylinder(level_start,level_end,0.05,col_level[i]))

## The energy level axis

In [28]:
axis = [Cylinder([L[0]/2+δ+0.0000,-2.5,-L[2]/2+1.5],[L[0]/2+δ+0.0000,9,-L[2]/2+1.5],0.05,col_level0)]
axis.append(Cone([L[0]/2+δ+0.0000,9,-L[2]/2+1.5],0.2,[L[0]/2+δ+0.0000,9.5,-L[2]/2+1.5],0.0,col_level0))

## Try to draw a thermometer

In [29]:
therm = [Sphere([L[0]/2+δ+0.05,-1,4],0.6,col_hot)]
therm.append(Cylinder([L[0]/2+δ+0.00,-1,4],[L[0]/2+δ+0.00,2,4],0.3,col_hot))
therm.append(Cylinder([L[0]/2+δ+0.00,2,4],[L[0]/2+δ+0.00,8,4],0.3,col_cold))
therm.append(Sphere([L[0]/2+δ+0.00,8,4],0.3,col_cold))

therm = [Merge(*therm)]

## Render the image

In [32]:
obj = [bg] + lights + box + sphere + cylinder + cone + levels + axis + therm
scene = Scene(camera=cam,objects=obj,included=included)
#scene.render('ipython', width=400, height=200,remove_temp=False)

size = 9600

scene.render(filename, width=size, height=size,quality=11,antialiasing=0.2,output_alpha=True,
             remove_temp=True)

## Autocrop the image

In [34]:
# autocrop the image
Image.MAX_IMAGE_PIXELS = 1000000000                                                                                              
image = Image.open(filename)
cropped = image.crop(image.getbbox())
cropped.save(filename)